In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
import pickle
import pandas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.svm import SVR
import itertools
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score
import numpy as nps
from itertools import combinations

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
##### only for testing the new ponds ################


import numpy as np
########### define training features and reset index####################################


def train_inf(train,test,param,scale):
  ph=[]
  sal=[]
  amm=[]
  mag=[]
  cal=[]
  hard=[]
  alk=[]
  carb=[]
  bicarb=[]
  print("==================")
  print(param)
  print("==================")
  print()
  df_train = train
  df_test = test

  df_train = df_train[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12", "pH", "Salinity", "Carbonate", "BiCarbonate",\
                       "Alkalinity", "Hardness","Calcium", "Magnesium","Ammonia"]]


  df_train = df_train.reset_index()

  df_test = df_test[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12", "pH", "Salinity", "Carbonate", "BiCarbonate",\
                       "Alkalinity", "Hardness","Calcium", "Magnesium","Ammonia"]]

  df_test = df_test.reset_index()
  print('------------------------------------------------------------------')
  #print(df_test.loc[0].dtypes)
  ###################get the index val #########################
  minimum = df_train[param].min()
  baseline_index = np.where([df_train[param]==minimum])[1][0]
  print('baseline val : ', minimum)
  print('baseline_index : ', baseline_index)
  print("___________________________________")



  ##################drop the baseline row and subtract that from training data and val data##########################
  baseline = df_train.loc[baseline_index]
  df_train = df_train.drop(baseline_index)

  df_train = df_train-baseline

  df_test = df_test-baseline


  xTrain = df_train[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12",]]

  yTrain = df_train[[param]]/int(scale)
  if param=='Ammonia':
   # df_test = df_test[df_test['Ammonia'].notna()]
    yTrain = df_train[[param]]*int(scale)

  xTest = df_test[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12"]]
  yTest = df_test[[param]]

  #model = pickle.load(open('/content/{}.sav'.format(params[num]), 'rb'))
  model  = SVR(kernel='rbf', C=1.0, epsilon=0.2)
  model.fit(xTrain, yTrain)

  pred = model.predict(xTest)

  prediction = pred*int(scale)
  if param=='Ammonia':
    prediction = pred/int(scale)


  yTest = yTest + minimum
  prediction = prediction + minimum


  ##### to save the models
  filename = '/content/{}.sav'.format(param)
  pickle.dump(model, open(filename, 'wb'))

  print("Test results")
  print("___________________________")
  ### uncomment the below code if GT values are present

  print("MAE", mean_absolute_error(yTest, prediction))
  # # print("MSE", mean_squared_error(yTest, prediction))
  # # print("RMSE", mean_squared_error(yTest, prediction, squared=False))
  print("MAPE", mean_absolute_percentage_error(yTest, prediction))
  print("R2 Score", r2_score(yTest, prediction))

  print()
  print(print(' Predicted' ))
  for i,j in zip(list(yTest[param]),prediction):
    print(i,j)

  print("===========================================================================")
  print()
  return prediction, baseline, minimum

In [4]:
dataframe = pd.read_csv("/content/drive/MyDrive/WQA_data/dataframe-tve.csv")
print("Total samples", len(dataframe))
dataframe = dataframe[dataframe["Coincidence"] == 0]
dataframe = dataframe[dataframe["Salinity"]>20]
print(len(dataframe))
trainingset = dataframe
# validationset = dataframe[dataframe["Date of Image"]=="2023-07-09"]
# trainingset = dataframe.drop(validationset.index)

Total samples 179
56


In [5]:
max(trainingset['Alkalinity'])

200

In [6]:
validationset = pd.read_csv('/content/drive/MyDrive/WQA_data/testset.csv')

validationset['pH'] = [8.0,8.1,8.1,7.8,7.5]
validationset['Salinity'] = [30,30,25,20,22]
validationset['Ammonia'] = [0.07,0.03,0.04,0.04,0.08]
validationset['Carbonate'] = [0,10,10,0,0]
validationset['BiCarbonate'] = [150,200,250,100,110]
validationset['Alkalinity'] = [150,200,250,100,110]
validationset['Hardness'] = [6800,6960,6500,7100,7500]
validationset['Calcium'] = [684,688,644,1080,1102]
validationset['Magnesium'] = [1232,1262,1126,1069,1121]


In [7]:
validationset

,B1,B10,B11,B12,B2,B3,B4,B5,B6,B7,...,PondID,pH,Salinity,Ammonia,Carbonate,BiCarbonate,Alkalinity,Hardness,Calcium,Magnesium
0,1209,96,691,437,954,1030,735,967,884,932,...,BR_P1,8.0,30,0.07,0,150,150,6800,684,1232
1,1212,81,1029,623,933,834,590,884,1423,1755,...,BR_P2,8.1,30,0.03,10,200,200,6960,688,1262
2,1198,102,921,532,1000,988,715,995,1026,1260,...,BR_P3,8.1,25,0.04,10,250,250,6500,644,1126
3,2110,360,1342,958,1730,1556,1417,1502,1862,2148,...,M_P1,7.8,20,0.04,0,100,100,7100,1080,1069
4,2123,358,1340,976,1746,1605,1416,1492,1850,2141,...,M_P2,7.5,22,0.08,0,110,110,7500,1102,1121


In [8]:
bands = ["B1", "B2", "B3","B4", "B5",  "B6", "B7", "B8", "B8A", "B9", "B11", "B12"]
parameters = ["pH", "Salinity", "Carbonate", "BiCarbonate",\
                       "Alkalinity", "Hardness","Calcium", "Magnesium","Ammonia"]
X_train, X_test, y_train, y_test = train_test_split(trainingset[bands], trainingset[parameters], test_size=0.33,random_state=42)

In [9]:
X_train[parameters] = y_train
X_test[parameters] = y_test
trainingset,validationset = X_train,X_test

In [10]:
ph,ph_baseline, ph_minimum =train_inf(trainingset,validationset,'pH',1)
print(ph_baseline, ph_minimum)
sal,sal_baseline, sal_minimum=train_inf(trainingset,validationset,'Salinity',10)
print(sal_baseline, sal_minimum)
amm,amm_baseline, amm_minimum =train_inf(trainingset,validationset,'Ammonia',100)
print(amm_baseline, amm_minimum)
mag,mag_baseline, mag_minimum = train_inf(trainingset,validationset,'Magnesium',1000)
print(mag_baseline, mag_minimum)
cal,cal_baseline, cal_minimum =train_inf(trainingset,validationset,'Calcium',1000)
print(cal_baseline, cal_minimum)
carb,carb_baseline, carb_minimum=train_inf(trainingset,validationset,'Carbonate',100)
print(carb_baseline,carb_minimum)
bicarb,bicarb_baseline, bicarb_minimum=train_inf(trainingset,validationset,'BiCarbonate',100)
print(bicarb_baseline, bicarb_minimum)
hard,hard_baseline, hard_minimum=train_inf(trainingset,validationset,'Hardness',1000)
print(hard_baseline, hard_minimum)
alk,alk_baseline, alk_minimum=train_inf(trainingset,validationset,'Alkalinity',100)
print(alk_baseline, alk_minimum)

# pH =  0.3238, 0.0404
# Salinity =  2.5328, 0.0643
# Ammonia = 0.03262, 0.4457
#mag = 323.5788, 0.2513
#cal = 91.0132, 0.1686
#carb = 15.3062, 4.0716
#bicarb = 34.7371, 0.2287
#hard = 960.6905, 0.1233
#alk = 23.9199, 0.1406
# sal = [1778.0,1683.0,1891.0,2268.0,2457.0,2654.0,2960.0,2793.0,3142.0,682.0,4002.0,3492.0]
# amm = [1461.0,1253.0,1154.0,919.0,1025.0,1059.0,1171.0,920.0,1188.0,165.0,980.0,612.0]
# mag =[1403.00,975.00,800.00,628.00,818.00,738.00,839.00,579.00,797.00,135.00,744.00,336.00]
# cal = [1551.00,1256.00,1189.00,912.00,1198.00,923.00,985.00,731.00,880.00,230.00,811.00,575.00]
# carb = [1672.00,1320.00,1195.00,945.00,1090.00,1003.00,1026.00,760.00,900.00,145.00,639.00,454.00]
# bicarb = [1497.0,1187.0,1055.0,809.0,926.0,862.0,923.0,711.0,942.0,160.0,789.0,481.0]
# hard = [1497.0,1187.0,1055.0,809.0,926.0,862.0,923.0,711.0,942.0,160.0,789.0,481.0]
# alk = [1403.00,975.00,800.00,628.00,818.00,738.00,839.00,579.00,797.00,135.00,744.00,336.00]


pH

------------------------------------------------------------------
baseline val :  7.5
baseline_index :  7
___________________________________
Test results
___________________________
MAE 0.24310744349938776
MAPE 0.030211576447264955
R2 Score 0.5263368221633086

 Predicted
None
8.4 8.399300020251351
8.6 8.38720677942306
7.6 8.016006341630428
8.4 8.345085368865751
8.6 8.382907234506426
8.3 8.171884191289772
7.9 8.000798550655599
7.7 8.000793261767521
7.7 8.262420123113996
8.5 8.370239055717834
7.7 8.031993604331856
8.7 8.405236939869926
7.6 8.00688492345708
7.7 8.0073138760036
8.6 8.337237311659354
8.4 8.2806882691531
8.6 8.401996410492936
8.6 8.313262264666465
8.1 8.387874591424264

index            90.00
B1             1869.00
B2             1548.00
B3             1403.00
B4             1232.00
B5             1351.00
B6             1298.00
B7             1406.00
B8             1173.00
B8A            1355.00
B9              304.00
B11            1012.00
B12             721.00
pH   

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

Test results
___________________________
MAE 138.05713526459053
MAPE 0.279742529882682
R2 Score 0.6162760170409709

 Predicted
None
388.0 507.00669679048565
384.0 521.3130907598522
874.0 776.1279278222992
466.0 479.37945066996826
352.0 518.1790666292367
520.0 582.0480055813564
902.0 738.991372342244
908.0 766.2552735634489
566.0 585.2151005048463
302.0 514.3022453922179
1126.0 754.4616043728693
488.0 556.054444237415
924.0 766.2341008975299
924.0 776.2263267451838
336.0 495.824756326232
368.0 579.8122581173607
354.0 545.8197711002106
378.0 517.6754347801161
520.0 477.2481451185025

index            52.00
B1             1543.00
B2             1268.00
B3             1253.00
B4              971.00
B5             1159.00
B6             1052.00
B7             1046.00
B8              874.00
B8A            1058.00
B9              293.00
B11             849.00
B12             603.00
pH                8.30
Salinity         35.00
Carbonate        20.00
BiCarbonate    1150.00
Alkalinity      170.

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [11]:
##### only for testing the new ponds ################


import numpy as np
########### define training features and reset index####################################


def test_inf(param,test_data):


  pH = [1584.00, 1376.00, 1321.0, 1296.00,1437.00,1501.00,1639.00,\
                  1483.00,1574.00,241.00,1097.00,666.00,7.5,1]

  Salinity = [1778.0,1683.0,1891.0,2268.0,2457.0,2654.0,2960.0,2793.0,\
         3142.0,682.0,4002.0,3492.0,21,10]
  Ammonia = [1461.0,1253.0,1154.0,919.0,1025.0,1059.0,1171.0,920.0,\
         1188.0,165.0,980.0,612.0,0.0,100]
  Magnesium =[1403.00,975.00,800.00,628.00,818.00,738.00,839.00,579.00,\
        797.00,135.00,744.00,336.00,603,1000]
  Calcium = [1551.00,1256.00,1189.00,912.00,1198.00,923.00,985.00,731.00,\
         880.00,230.00,811.00,575.00,300,1000]
  Carbonate = [1672.00,1320.00,1195.00,945.00,1090.00,1003.00,1026.00,\
          760.00,900.00,145.00,639.00,454.00,0,100]
  Bicarbonate = [1497.0,1187.0,1055.0,809.0,926.0,862.0,923.0,711.0,\
            942.0,160.0,789.0,481.0,90,100]
  Hardness = [1497.0,1187.0,1055.0,809.0,926.0,862.0,923.0,711.0,942.0,\
          160.0,789.0,481.0,4500,1000]
  Alkalinity = [1403.00,975.00,800.00,628.00,818.00,738.00,839.00,579.00,\
         797.00,135.00,744.00,336.00,120,100]

  baselines = {'pH':pH,'Salinity':Salinity,'Ammonia':Ammonia,\
               'Magnesium':Magnesium,'Calcium':Calcium,'Carbonate':Carbonate,\
                'Bicarbonate':Bicarbonate,'Hardness':Hardness,'Alkalinity':Alkalinity}

  baseline = pd.Series(baselines[param][:-2], index =["B1", "B2", "B3", "B4",\
                  'B5','B6','B7','B8','B8A','B9','B11','B12'])

  minimum = baselines[param][-2]
  scale = baselines[param][-1]
  test_data = test_data[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12"]]
  test_data = test_data.subtract(baseline, axis = 1)

  model = pickle.load(open('/content/{}.sav'.format(param), 'rb'))
  pred = model.predict(test_data)
  prediction = pred*int(scale)
  if param=='Ammonia':
    prediction = pred/int(scale)
  prediction = prediction + minimum

  return prediction

In [12]:
pred= test_inf('pH',validationset)
print(pred)

[8.35790144 8.33571419 7.96936213 8.27418145 8.31287836 8.20941366
 8.05308709 8.00776655 8.26386583 8.31149976 8.06155068 8.34560413
 7.98283546 7.96275636 8.27813816 8.23547818 8.33422531 8.252759
 8.38538161]


In [13]:
##### only for testing the new ponds ################


import numpy as np
########### define training features and reset index####################################


def test_inf(model,param,test_data,baseline_bands,baseline_val,scale):

  df_test = test

  df_test = df_test[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12"]]

  df_test = df_test.reset_index()
  print('------------------------------------------------------------------')
  print(df_test.loc[0].dtypes)
  ##################get the index val #########################
  baseline_index = np.where([df_train[param]==minimum])[1][0]
  print('baseline val : ', minimum)
  print('baseline_index : ', baseline_index)
  print("___________________________________")



  ##################drop the baseline row and subtract that from training data and val data##########################
  # baseline = df_train.loc[baseline_index]
  # df_train = df_train.drop(baseline_index)

  # df_train = df_train-baseline

  df_test = df_test-baseline


  # xTrain = df_train[["B1", "B2", "B3","B4", "B5", \
  #                      "B6", "B7", "B8", "B8A", "B9", "B11", "B12",]]

  # yTrain = df_train[[param]]/int(scale)
  # if param=='Ammonia':
  #   df_test = df_test[df_test['Ammonia'].notna()]
  #   yTrain = df_train[[param]]*int(scale)

  # xTest = df_test[["B1", "B2", "B3","B4", "B5", \
  #                      "B6", "B7", "B8", "B8A", "B9", "B11", "B12",]]
  # yTest = df_test[[param]]

  model = pickle.load(open('/content/{}.sav'.format(params[num]), 'rb'))
  #model  = SVR(kernel='rbf', C=1.0, epsilon=0.2)
  # model.fit(xTrain, yTrain)

  pred = model.predict(df_test)

  prediction = pred*int(scale)
  if param=='Ammonia':
    prediction = pred/int(scale)


  #yTest = yTest + minimum
  prediction = prediction + baseline_val


  # ##### to save the models
  # filename = '/content/{}.sav'.format(param)
  # pickle.dump(model, open(filename, 'wb'))

  # print("Test results")
  # print("___________________________")


  # print("===========================================================================")
  # print()
  return prediction

In [14]:
testingset_tup = pd.read_csv('/content/Tup_TS.csv')
testingset_vav = pd.read_csv('/content/Vav_TS.csv')


FileNotFoundError: ignored

In [ ]:
ph_tup =test_inf(trainingset,testingset_tup,'pH',1)
sal_tup=test_inf(trainingset,testingset_tup,'Salinity',10)
amm_tup =test_inf(trainingset,testingset_tup,'Ammonia',100)

testingset_tup['predicted_pH'] = ph_tup
testingset_tup['predicted_salinity'] = sal_tup
testingset_tup['predicted_ammonia'] = amm_tup

ph_vav =test_inf(trainingset,testingset_vav,'pH',1)
sal_vav =test_inf(trainingset,testingset_vav,'Salinity',10)
amm_vav =test_inf(trainingset,testingset_vav,'Ammonia',100)

testingset_vav['predicted_pH'] = ph_vav
testingset_vav['predicted_salinity'] = sal_vav
testingset_vav['predicted_ammonia'] = amm_vav

In [ ]:
ponds = []
for i in range(len(testingset_tup)):
  ponds.append('T_'+testingset_tup['PondID'].loc[i].split('_')[1])

testingset_tup['pond'] = ponds

ponds = []
for i in range(len(testingset_vav)):
  ponds.append('T_'+testingset_vav['PondID'].loc[i].split('_')[1])

testingset_vav['pond'] = ponds

test_tup = testingset_tup[["PondID",	"predicted_pH",	"predicted_salinity",	"predicted_ammonia",	"pond"]].to_csv('/content/tupilipalem_ts.csv',index=False)
test_vav = testingset_vav[["PondID",	"predicted_pH",	"predicted_salinity",	"predicted_ammonia",	"pond"]].to_csv('/content/vavilipalem_ts.csv',index=False)



In [ ]:
testingset_eru = pd.read_csv('/content/Eru_TS.csv')


ph_eru =test_inf(trainingset,testingset_eru,'pH',1)
sal_eru=test_inf(trainingset,testingset_eru,'Salinity',10)
amm_eru =test_inf(trainingset,testingset_eru,'Ammonia',100)

testingset_eru['predicted_pH'] = ph_eru
testingset_eru['predicted_salinity'] = sal_eru
testingset_eru['predicted_ammonia'] = amm_eru

ponds = []
for i in range(len(testingset_eru)):
  ponds.append('E_'+testingset_eru['PondID'].loc[i].split('_')[0])

testingset_eru['pond'] = ponds


test_eru = testingset_eru[["PondID",	"predicted_pH",	"predicted_salinity",	"predicted_ammonia",	"pond"]].to_csv('/content/eruru_ts.csv',index=False)




In [ ]:
testingset_eru

In [ ]:
##### only for testing the new ponds ################


import numpy as np

########### define training features and reset index####################################


def train_inf_absolute(train,test,param,scale):
  ph=[]
  sal=[]
  amm=[]
  mag=[]
  cal=[]
  hard=[]
  alk=[]
  carb=[]
  bicarb=[]
  print("==================")
  print(param)
  print("==================")
  print()
  df_train = train
  df_test = test

  df_train = df_train[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12", "pH", "Salinity", "Carbonate", "BiCarbonate",\
                       "Alkalinity", "Hardness","Calcium", "Magnesium", "Ammonia"]]
  df_train = df_train.reset_index()

  df_test = df_test[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12", "pH", "Salinity", "Carbonate", "BiCarbonate",\
                       "Alkalinity", "Hardness","Calcium", "Magnesium", "Ammonia"]]

  ###################get the index val #########################
  minimum = df_train[param].min()
  baseline_index = np.where([df_train[param]==minimum])[1][0]
  print('baseline val : ', minimum)
  print('baseline_index : ', baseline_index)
  print("___________________________________")



  ##################drop the baseline row and subtract that from training data and val data##########################
  baseline = df_train.loc[baseline_index]
  df_train = df_train.drop(baseline_index)

  df_train = df_train
  df_test = df_test


  xTrain = df_train[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12"]]
  yTrain = df_train[[param]]/int(scale)
  if param=='Ammonia':
    df_test = df_test[df_test['Ammonia'].notna()]
    yTrain = df_train[[param]]*int(scale)

  xTest = df_test[["B1", "B2", "B3","B4", "B5", \
                       "B6", "B7", "B8", "B8A", "B9", "B11", "B12"]]
  yTest = df_test[[param]]

  #model = pickle.load(open('/content/{}.sav'.format(params[num]), 'rb'))
  model  = SVR(kernel='rbf', C=1.0, epsilon=0.2)
  model.fit(xTrain, yTrain)

  pred = model.predict(xTest)

  prediction = pred*int(scale)
  if param=='Ammonia':
    prediction = pred/int(scale)


  yTest = yTest
  prediction = prediction


  ##### to save the models
  # filename = '/content/{}.sav'.format(params[num])
  # pickle.dump(model, open(filename, 'wb'))

  print("Test results")
  print("___________________________")
  ### uncomment the below code if GT values are present

  print("MAE", mean_absolute_error(yTest, prediction))
  # # print("MSE", mean_squared_error(yTest, prediction))
  # # print("RMSE", mean_squared_error(yTest, prediction, squared=False))
  # print("MAPE", mean_absolute_percentage_error(yTest, prediction))
#   print("R2 Score", r2_score(yTest, prediction))

  print()
  print(print('Ground Truth - Predicted' ))
  for i,j in zip(list(yTest[param]),prediction):
    print(i, j)

  print("===========================================================================")
  print()
  return prediction

# toa

\## training baseline model coincidence = 0

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
import pickle
import pandas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.svm import SVR
import itertools
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score
import numpy as nps
from itertools import combinations

In [ ]:
kotatest = pd.read_csv('/content/kotatestnew.csv')
kotatest

In [ ]:
newtestdata = pd.read_csv('/content/newtestdata.csv')
newtestdata

In [ ]:
testtesttest = pd.read_csv('/content/drive/MyDrive/WQA_data/testtesttest.csv')
testtesttest

In [ ]:
dataframe = pd.read_csv("/content/drive/MyDrive/WQA_data/dataframe-tve.csv")
print("Total samples", len(dataframe))
#dataframe = dataframe[dataframe["Coincidence"]<3]


In [ ]:
#dataframe = dataframe[dataframe["Coincidence"]<3]
dataframe = dataframe[dataframe["Salinity"]>20]

In [ ]:
for l in dataframe["PondID"].unique():
    print("Samples for date ",l,": ",len(dataframe[dataframe["PondID"]==l]))

In [ ]:
dataframe.columns

In [ ]:
for l in dataframe["Date of Image"].unique():
    print("Samples for date ",l,": ",len(dataframe[dataframe["Date of Image"]==l]))

In [ ]:
### selecting 09-07-2023 for testing

validationset = dataframe[dataframe["Date of Image"]=="2023-07-09"]
trainingset = dataframe.drop(validationset.index)

In [ ]:
testtesttest = testtesttest.drop(testtesttest.index)

In [ ]:
testtesttest

In [ ]:
trainingset

In [ ]:
validationset

In [ ]:
ph_pred = [8.283518646989906,
           8.39827259582833,

           8.437517215870502,


           8.357304765423109,
           8.338181272065539,
          8.504697922084581,
           8.476198416646447,
           8.437434493930766,
           8.420368313222493,

          ]

ph_actual = [8.0 ,8.5 ,8.2  ,8.5 ,8.2 ,8.4 ,8.0  ,8.3 ,8.3 ]


sal_actual = [35.0,
40.0,
40.0,
40.0,
36.0,
35.0,
42.0,
40.0,
40.0,
40.0,
39.0]
sal_pred = [
 35.766733763215015,
 36.78327574116457,
 36.790536886494635,
 38.37429978113569,
 38.52655930503117,
 36.70495239515186,
 39.29663954283452,
 38.75871428895375,
 37.57778259804472,
 38.336066926496706,
 37.108181099216765]


amm_actual = [
    0.04,
0.05,
0.04,
0.05,
0.05,
0.04,
0.04
 ]

amm_pred =[
 0.03785122426086872,
 0.02922070951671172,
 0.03432068166529676,
 0.03229762133762642,
 0.03288830991278609,
 0.036672636438224104,
 0.029969836577737857]

In [ ]:

plt.figure(figsize=(9,5))
plt.scatter(np.arange(1, len(amm_pred)+1), amm_actual, label="Ground Truth")
plt.scatter(np.arange(1, len(amm_pred)+1), amm_pred,color = 'red')
plt.plot(np.arange(1, len(amm_pred)+1), amm_pred, color="red", label="Prediction")
plt.ylim(0.0,0.1)
plt.ylabel("Ammonia")
plt.xlabel("Sample Index")
plt.legend()
plt.legend( fontsize="8", loc ="lower right")
plt.grid()

In [ ]:

plt.figure(figsize=(9,5))
plt.scatter(np.arange(1, len(sal_pred)+1), sal_actual, label="Ground Truth")
plt.scatter(np.arange(1, len(sal_pred)+1), sal_pred,color = 'red')
plt.plot(np.arange(1, len(sal_pred)+1), sal_pred, color="red", label="Prediction")
plt.ylim(15,48)
plt.ylabel("Salinity")
plt.xlabel("Sample Index")
plt.legend()
plt.legend( fontsize="8", loc ="lower right")
plt.grid()

In [ ]:


plt.figure(figsize=(9,5))
plt.scatter(np.arange(1, len(ph_pred)+1), ph_actual, label="Ground Truth")
plt.scatter(np.arange(1, len(ph_pred)+1), ph_pred,color = 'red')
plt.plot(np.arange(1, len(ph_pred)+1), ph_pred, color="red", label="Prediction")
plt.ylim(6, 9)
plt.ylabel("pH")
plt.xlabel("Sample Index")
plt.legend()
plt.legend( fontsize="8", loc ="lower right")
plt.grid()

In [ ]:
ph_actual = [8.4,8.7,8.2]
ph_pred = [8.222875312028107,8.460432532556236,7.910295014017632]

sal_actual = [27.0,
40.0,
25.0]
sal_pred = [
 29.581858147672648,
 33.787225134101774,
 25.020637273885164]

amm_actual = [0.0 ,
0.01,
0.06]
amm_pred = [
0.032489317509076535,
 0.016459947872140838,
 0.061422380041368144]


In [ ]:
ph_actual = [8.4,8.7,8.2,8.0]
ph_pred = [8.222875312028107,8.460432532556236,7.910295014017632,8.0883445397316]

sal_actual = [27.0,
40.0,
25.0,
34.0]
sal_pred = [
 29.581858147672648,
 33.787225134101774,
 25.020637273885164,
 30.672317744182976]

amm_actual = [0.0 ,
0.01,
0.06]
amm_pred = [
0.032489317509076535,
 0.016459947872140838,
 0.061422380041368144]


In [ ]:

plt.figure(figsize=(9,5))
plt.scatter(np.arange(1, len(amm_pred)+1), amm_actual, label="Ground Truth")
plt.scatter(np.arange(1, len(amm_pred)+1), amm_pred,color = 'red')
plt.plot(np.arange(1, len(amm_pred)+1), amm_pred, color="red", label="Prediction")
plt.ylim(0.0,0.1)
plt.ylabel("Ammonia")
plt.xlabel("Sample Index")
plt.legend()
plt.legend( fontsize="8", loc ="lower right")
plt.grid()

In [ ]:

plt.figure(figsize=(9,5))
plt.scatter(np.arange(1, len(sal_pred)+1), sal_actual, label="Ground Truth",s=85)
plt.scatter(np.arange(1, len(sal_pred)+1), sal_pred,color = 'red')
plt.plot(np.arange(1, len(sal_pred)+1), sal_pred, color="red", label="Prediction")
plt.ylim(15,48)
plt.ylabel("Salinity")
plt.xlabel("Sample Index")
plt.legend()
plt.legend( fontsize="8", loc ="lower right")
plt.grid()

In [ ]:


plt.figure(figsize=(9,5))
plt.scatter(np.arange(1, len(ph_pred)+1), ph_actual, label="Ground Truth")
plt.scatter(np.arange(1, len(ph_pred)+1), ph_pred,color = 'red')
plt.plot(np.arange(1, len(ph_pred)+1), ph_pred, color="red", label="Prediction")
plt.ylim(6, 9)
plt.ylabel("pH")
plt.xlabel("Sample Index")
plt.legend()
plt.legend( fontsize="8", loc ="lower right")
plt.grid()

In [ ]:
validationset.columns

In [ ]:
testtesttest.columns

In [ ]:
dataframe = pd.read_csv("/content/drive/MyDrive/WQA_data/dataframe-tve.csv")
print("Total samples", len(dataframe))
#dataframe = dataframe[dataframe["Coincidence"]<3]
dataframe = dataframe[dataframe["Salinity"]>20]
print(len(dataframe))
validationset = dataframe[dataframe["Date of Image"]=="2023-07-09"]
trainingset = dataframe.drop(validationset.index)

In [ ]:
trainingset = trainingset[trainingset['Coincidence']==0]

In [ ]:
trainingset

In [ ]:
testtesttest

In [ ]:

#= testtesttest.loc[:3]
ocean = testtesttest.loc[3]

In [ ]:
ocean

In [ ]:
validationset =testtesttest
# trainingset = dataframe

In [ ]:
#valid set removed
ph = 0.19
sal = 3.14/2.93
amm = 0.25

#valid set included
ph =0.22
sal = 3.
amm = 0.25

In [ ]:

ocean = {'PondID':'Ocean', 'pH':0, 'Salinity':34, 'Carbonate':0, 'BiCarbonate':0, 'Alkalinity':0,
       'Hardness':0, 'Calcium':0, 'Magnesium':0, 'Ammonia':0, 'DateofReport':0,
       'Coincidence':0, 'Date of Image':0, 'B1':1884, 'B10':1576, 'B11':485, 'B12':337, 'B2':1576, 'B3':1220,
       'B4':910, 'B5':818, 'B6':780, 'B7':798, 'B8':688, 'B8A':750, 'B9':179}

saltrain = trainingset.append(ocean, ignore_index = True)
saltrain


In [ ]:
validationset

In [ ]:
saltest = testtesttest.loc[:2]

In [ ]:
saltest

In [ ]:
validationset = saltest

In [ ]:
pH = trainingset['pH']
sal = trainingset['Salinity']
amm = trainingset['Ammonia']



In [ ]:
plt.hist(trainingset['pH'])

In [ ]:
trainingset.to_csv('/content/train_report.csv',index=False)

In [ ]:
plt.figure(figsize=(9,5))
plt.scatter(np.arange(1, len(pH)+1), pH, label="Ground Truth Distribution",s = 30)
plt.ylim(7.0,9.5)
plt.ylabel("pH")
plt.xlabel("Sample Index")
plt.legend()
plt.legend( fontsize="8", loc ="lower right")
plt.grid()

In [ ]:
validationset = pd.read_csv('/content/tt.csv')
validationset

In [ ]:
dataframe = pd.read_csv("./dataframe-tve.csv")
print("Total samples", len(dataframe))
#dataframe = dataframe[dataframe["Coincidence"]<3]
dataframe = dataframe[dataframe["Salinity"]>20]
print(len(dataframe))
#validationset = dataframe[dataframe["Date of Image"]=="2023-07-09"]
#trainingset = dataframe.drop(validationset.index)

In [ ]:
test = pd.read_csv('/content/avanti_rest1.csv')
test.columns
validationset = test

In [ ]:
prev = pd.read_csv('/content/dataframe-tve.csv')
prev.columns

In [ ]:
trainingset = pd.read_csv('/content/WQIModelTrain.csv')
validationset = pd.read_csv('/content/WQIModelTest.csv')

In [ ]:
validationset.columns
validationset = validationset.drop(['Unnamed: 0'],axis = 1)

In [ ]:
trainingset = trainingset.drop(['Unnamed: 0'],axis = 1)

In [ ]:
trainingset = dataframe[dataframe["Salinity"]>20]

In [ ]:
validationset = pd.read_csv('/content/test-sentinel-new.csv')

In [ ]:
validationset = validationset[validationset["Salinity"]>20]
validationset.dtypes

In [ ]:
valid_amm = validationset[validationset['Ammonia'].notna()]
valid_amm

In [ ]:
convert_dict = {'Ammonia': float
                }
valid_amm = valid_amm.astype(convert_dict)


In [ ]:
np.where(valid_amm=='Nill')

In [ ]:
len(amm)

In [ ]:
release = valid_amm[['village_name','farmer_name','pond_name','pH','Salinity','Ammonia']]
release['predicted_pH'] = ph
release['predicted_Salinity'] = sal
release['predicted_Ammonia']= amm
release.to_csv('/content/test_results.csv',index =False)

In [ ]:
release

In [ ]:
validationset

In [ ]:
validationset.columns

In [ ]:
len(sal)

In [ ]:
ph =train_inf(trainingset,valid_amm,'pH',1)
sal=train_inf(trainingset,valid_amm,'Salinity',10)
amm =train_inf(trainingset,valid_amm,'Ammonia',100)
# mag = train_inf(trainingset,validationset,'Magnesium',1000)
# cal =train_inf(trainingset,validationset,'Calcium',1000)
# carb=train_inf(trainingset,validationset,'Carbonate',100)
# bicarb=train_inf(trainingset,validationset,'BiCarbonate',100)
# hard=train_inf(trainingset,validationset,'Hardness',1000)
# alk=train_inf(trainingset,validationset,'Alkalinity',100)
pH = 0.2271
sal =  4.6838
amm = 0.0175

In [ ]:
ph =train_inf(trainingset,validationset,'pH',1)
sal=train_inf(saltrain,validationset,'Salinity',10)
amm =train_inf(trainingset,validationset,'Ammonia',100)
mag = train_inf(trainingset,validationset,'Magnesium',1000)
cal =train_inf(trainingset,validationset,'Calcium',1000)
carb=train_inf(trainingset,validationset,'Carbonate',100)
bicarb=train_inf(trainingset,validationset,'BiCarbonate',100)
hard=train_inf(trainingset,validationset,'Hardness',1000)
alk=train_inf(trainingset,validationset,'Alkalinity',100)

In [ ]:
ph =train_inf(trainingset,validationset,'pH',1)
sal=train_inf(trainingset,validationset,'Salinity',10)
amm =train_inf(trainingset,validationset,'Ammonia',100)
mag = train_inf(trainingset,validationset,'Magnesium',1000)
cal =train_inf(trainingset,validationset,'Calcium',1000)
carb=train_inf(trainingset,validationset,'Carbonate',100)
bicarb=train_inf(trainingset,validationset,'BiCarbonate',100)
hard=train_inf(trainingset,validationset,'Hardness',1000)
alk=train_inf(trainingset,validationset,'Alkalinity',100)

In [ ]:
ph =train_inf(trainingset,validationset,'pH',1)
sal=train_inf(trainingset,validationset,'Salinity',10)
amm =train_inf(trainingset,validationset,'Ammonia',100)
mag = train_inf(trainingset,validationset,'Magnesium',1000)
cal =train_inf(trainingset,validationset,'Calcium',1000)
carb=train_inf(trainingset,validationset,'Carbonate',100)
bicarb=train_inf(trainingset,validationset,'BiCarbonate',100)
hard=train_inf(trainingset,validationset,'Hardness',1000)
alk=train_inf(trainingset,validationset,'Alkalinity',100)

In [ ]:
ph =train_inf(trainingset,validationset,'pH',1)
sal=train_inf(trainingset,validationset,'Salinity',10)
amm =train_inf(trainingset,validationset,'Ammonia',100)
mag = train_inf(trainingset,validationset,'Magnesium',1000)
cal =train_inf(trainingset,validationset,'Calcium',1000)
carb=train_inf(trainingset,validationset,'Carbonate',100)
bicarb=train_inf(trainingset,validationset,'BiCarbonate',100)
hard=train_inf(trainingset,validationset,'Hardness',1000)
alk=train_inf(trainingset,validationset,'Alkalinity',100)

In [ ]:
ph =train_inf_absolute(trainingset,validationset,'pH',1)
sal=train_inf_absolute(trainingset,validationset,'Salinity',10)
amm =train_inf_absolute(trainingset,validationset,'Ammonia',100)
mag = train_inf_absolute(trainingset,validationset,'Magnesium',1000)
cal =train_inf_absolute(trainingset,validationset,'Calcium',1000)
carb=train_inf_absolute(trainingset,validationset,'Carbonate',100)
bicarb=train_inf_absolute(trainingset,validationset,'BiCarbonate',100)
hard=train_inf_absolute(trainingset,validationset,'Hardness',1000)
alk=train_inf_absolute(trainingset,validationset,'Alkalinity',100)

## Only zero coincidence

In [ ]:
dataframe = pd.read_csv("/content/drive/MyDrive/WQA_data/dataframe-tve.csv")
# dataframe = dataframe[dataframe["Coincidence"]=0]
dataframe = dataframe[dataframe["Salinity"]>20]
print("Total samples", len(dataframe))

In [ ]:
for l in dataframe["Date of Image"].unique():
    print("Samples for date ",l,": ",len(dataframe[dataframe["Date of Image"]==l]))

In [ ]:
### selecting 09-07-2023 for testing

validationset = dataframe[dataframe["Date of Image"]=="2023-07-09"]
trainingset = dataframe.drop(validationset.index)

In [ ]:
trainingset

In [ ]:
validationset = pd.read_csv("./kotatestnew.csv")

In [ ]:
validationset

In [ ]:
ph =train_inf(trainingset,validationset,'pH',1)
sal=train_inf(trainingset,validationset,'Salinity',10)
amm =train_inf(trainingset,validationset,'Ammonia',100)
mag = train_inf(trainingset,validationset,'Magnesium',1000)
cal =train_inf(trainingset,validationset,'Calcium',1000)
carb=train_inf(trainingset,validationset,'Carbonate',100)
bicarb=train_inf(trainingset,validationset,'BiCarbonate',100)
hard=train_inf(trainingset,validationset,'Hardness',1000)
alk=train_inf(trainingset,validationset,'Alkalinity',100)

In [ ]:
ph =train_inf_absolute(trainingset,validationset,'pH',1)
sal=train_inf_absolute(trainingset,validationset,'Salinity',10)
amm =train_inf_absolute(trainingset,validationset,'Ammonia',100)
mag = train_inf_absolute(trainingset,validationset,'Magnesium',1000)
cal =train_inf_absolute(trainingset,validationset,'Calcium',1000)
carb=train_inf_absolute(trainingset,validationset,'Carbonate',100)
bicarb=train_inf_absolute(trainingset,validationset,'BiCarbonate',100)
hard=train_inf_absolute(trainingset,validationset,'Hardness',1000)
alk=train_inf_absolute(trainingset,validationset,'Alkalinity',100)